<h1 style="text-align:center;"><b>Laboratorio 9</b></h1>
<h3 style="text-align:center;">Alina Carías (22539), Ariela Mishaan (22052), Marcos Díaz (221102) </h3>

https://github.com/ArielaMishaanCohen/17-Lab-9-SparkML.git

# Librerías

In [1]:
import pandas as pd
import pyspark 
from pyspark import SparkContext
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder.appName("Lab 9").getOrCreate()

25/10/09 14:50:49 WARN Utils: Your hostname, Arielas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.100.8.12 instead (on interface en0)
25/10/09 14:50:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/09 14:51:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Análisis Exploratorio Avanzado y Segmentación

## Preparación

In [3]:
df_fallecidos = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('ine/fallecidos_junto.csv')
df_hechos = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('ine/hechos_junto.csv')
df_vehiculos = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('ine/vehiculos_junto.csv')

### Fallecidos

In [4]:
df_fallecidos.schema

StructType([StructField('núm_corre0', DoubleType(), True), StructField('año_ocu', IntegerType(), True), StructField('mes_ocu', StringType(), True), StructField('día_ocu', IntegerType(), True), StructField('día_sem_ocu', IntegerType(), True), StructField('hora_ocu', IntegerType(), True), StructField('g_hora', IntegerType(), True), StructField('g_hora_5', IntegerType(), True), StructField('depto_ocu', IntegerType(), True), StructField('mupio_ocu', StringType(), True), StructField('área_geo_ocu', DoubleType(), True), StructField('zona_ocu', IntegerType(), True), StructField('sexo_per', IntegerType(), True), StructField('edad_per', IntegerType(), True), StructField('mayor_menor', IntegerType(), True), StructField('g_edad_80ymás', StringType(), True), StructField('g_edad_60ymás', IntegerType(), True), StructField('edad_quinquenales', StringType(), True), StructField('fall_les', IntegerType(), True), StructField('int_o_noint', IntegerType(), True), StructField('tipo_veh', IntegerType(), True

In [5]:
df_fallecidos.show(10)

+----------+-------+-------+-------+-----------+--------+------+--------+---------+---------+------------+--------+--------+--------+-----------+-------------+-------------+-----------------+--------+-----------+--------+---------+---------+----------+------------+--------+-----------+-----------+---------+
|núm_corre0|año_ocu|mes_ocu|día_ocu|día_sem_ocu|hora_ocu|g_hora|g_hora_5|depto_ocu|mupio_ocu|área_geo_ocu|zona_ocu|sexo_per|edad_per|mayor_menor|g_edad_80ymás|g_edad_60ymás|edad_quinquenales|fall_les|int_o_noint|tipo_veh|marca_veh|color_veh|modelo_veh|g_modelo_veh|tipo_eve|Núm_corre26|zona_ciudad|num_corre|
+----------+-------+-------+-------+-----------+--------+------+--------+---------+---------+------------+--------+--------+--------+-----------+-------------+-------------+-----------------+--------+-----------+--------+---------+---------+----------+------------+--------+-----------+-----------+---------+
|       1.0|   2015|      1|      1|          4|      16|     3|       2|

25/10/09 14:51:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/09 14:51:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: núm_corre, año_ocu, mes_ocu, día_ocu, día_sem_ocu, hora_ocu, g_hora, g_hora_5, depto_ocu, mupio_ocu, área_geo_ocu, zona_ocu, sexo_per, edad_per, mayor_menor, g_edad_80ymás, g_edad_60ymás, edad_quinquenales, fall_les, int_o_noint, tipo_veh, marca_veh, color_veh, modelo_veh, g_modelo_veh, tipo_eve, Núm_corre, zona_ciudad, num_corre
 Schema: núm_corre0, año_ocu, mes_ocu, día_ocu, día_sem_ocu, hora_ocu, g_hora, g_hora_5, depto_ocu, mupio_ocu, área_geo_ocu, zona_ocu, sexo_per, edad_per, mayor_menor, g_edad_80ymás, g_edad_60ymás, edad_quinquenales, fall_les, int_o_noint, tipo_veh, marca_veh, color_veh, modelo_veh, g_modelo_veh, tipo_eve, Núm_corre26, zona_ciudad, num_corre
Expected: núm_corre0 but found: núm_corre


In [6]:
df_fallecidos.count()

91367

### Hechos

In [7]:
df_hechos.schema

StructType([StructField('núm_corre', DoubleType(), True), StructField('año_ocu', IntegerType(), True), StructField('mes_ocu', StringType(), True), StructField('día_ocu', IntegerType(), True), StructField('hora_ocu', StringType(), True), StructField('día_sem_ocu', StringType(), True), StructField('g_hora', StringType(), True), StructField('g_hora_5', StringType(), True), StructField('depto_ocu', StringType(), True), StructField('mupio_ocu', StringType(), True), StructField('área_geo_ocu', DoubleType(), True), StructField('zona_ocu', StringType(), True), StructField('tipo_veh', StringType(), True), StructField('marca_veh', StringType(), True), StructField('color_veh', StringType(), True), StructField('modelo_veh', StringType(), True), StructField('tipo_eve', StringType(), True), StructField('g_modelo_veh', StringType(), True)])

In [8]:
df_hechos.show(10)

+---------+-------+-------+-------+--------+-----------+------+--------+---------+---------+------------+--------+--------+---------+---------+----------+--------+------------+
|núm_corre|año_ocu|mes_ocu|día_ocu|hora_ocu|día_sem_ocu|g_hora|g_hora_5|depto_ocu|mupio_ocu|área_geo_ocu|zona_ocu|tipo_veh|marca_veh|color_veh|modelo_veh|tipo_eve|g_modelo_veh|
+---------+-------+-------+-------+--------+-----------+------+--------+---------+---------+------------+--------+--------+---------+---------+----------+--------+------------+
|      1.0|   2015|      1|      1|      16|          4|     3|       2|        1|      101|         1.0|       1|       4|       21|        5|      2011|       1|           5|
|      2.0|   2015|      1|      1|      22|          4|     4|       3|        1|      101|         1.0|      16|       4|       21|        5|      9999|       1|           6|
|      3.0|   2015|      1|      1|       2|          4|     1|       1|        1|      101|         1.0|       7| 

In [9]:
df_hechos.count()

64784

### Vehículos

In [ ]:
df_vehiculos.schema

StructType([StructField('año_ocu', IntegerType(), True), StructField('num_corre', IntegerType(), True), StructField('mes_ocu', IntegerType(), True), StructField('día_ocu', IntegerType(), True), StructField('día_sem_ocu', IntegerType(), True), StructField('hora_ocu', IntegerType(), True), StructField('g_hora', IntegerType(), True), StructField('g_hora_5', IntegerType(), True), StructField('depto_ocu', IntegerType(), True), StructField('mupio_ocu', StringType(), True), StructField('área_geo_ocu', DoubleType(), True), StructField('zona_ocu', IntegerType(), True), StructField('sexo_per', IntegerType(), True), StructField('edad_per', IntegerType(), True), StructField('mayor_menor', IntegerType(), True), StructField('g_edad_80ymás', StringType(), True), StructField('g_edad_60ymás', IntegerType(), True), StructField('edad_quinquenales', StringType(), True), StructField('estado_con', IntegerType(), True), StructField('tipo_veh', IntegerType(), True), StructField('marca_veh', IntegerType(), Tru

In [ ]:
df_vehiculos.show(10)

+-------+---------+-------+-------+-----------+--------+------+--------+---------+---------+------------+--------+--------+--------+-----------+-------------+-------------+-----------------+----------+--------+---------+---------+----------+------------+--------+-----------+
|año_ocu|num_corre|mes_ocu|día_ocu|día_sem_ocu|hora_ocu|g_hora|g_hora_5|depto_ocu|mupio_ocu|área_geo_ocu|zona_ocu|sexo_per|edad_per|mayor_menor|g_edad_80ymás|g_edad_60ymás|edad_quinquenales|estado_con|tipo_veh|marca_veh|color_veh|modelo_veh|g_modelo_veh|tipo_eve|zona_ciudad|
+-------+---------+-------+-------+-----------+--------+------+--------+---------+---------+------------+--------+--------+--------+-----------+-------------+-------------+-----------------+----------+--------+---------+---------+----------+------------+--------+-----------+
|   2015|        1|      1|      1|          4|      16|     3|       2|        1|      101|         1.0|       1|       2|      24|          1|            3|            3|

In [10]:
df_vehiculos.count()

97761

In [12]:

# Agrupar por esas columnas y contar los vehículos (filas)
df_accidentes = df_hechos.groupBy(['año_ocu']) \
    .agg(F.count('*').alias('vehiculos_involucrados')) \
    .orderBy(F.desc('vehiculos_involucrados'))  

df_accidentes.show()

+-------+----------------------+
|año_ocu|vehiculos_involucrados|
+-------+----------------------+
|   2023|                  8218|
|   2021|                  8153|
|   2016|                  7964|
|   2022|                  7924|
|   2019|                  7047|
|   2015|                  6854|
|   2018|                  6395|
|   2020|                  6350|
|   2017|                  5879|
+-------+----------------------+



### Dataset combinado

In [13]:
df_hechos

DataFrame[núm_corre: double, año_ocu: int, mes_ocu: string, día_ocu: int, hora_ocu: string, día_sem_ocu: string, g_hora: string, g_hora_5: string, depto_ocu: string, mupio_ocu: string, área_geo_ocu: double, zona_ocu: string, tipo_veh: string, marca_veh: string, color_veh: string, modelo_veh: string, tipo_eve: string, g_modelo_veh: string]

#### Con vehículos

In [28]:
# Columnas que identifican un accidente
columnas_vehiculos = [
    'año_ocu', 'mes_ocu', 'día_ocu', 'hora_ocu',
    'depto_ocu', 'mupio_ocu', 'zona_ocu', 'día_sem_ocu', 'tipo_eve', 
    'g_hora', 'g_hora_5'
]

# Agrupar por esas columnas y contar los vehículos (filas)
vehiculos = df_vehiculos.groupBy(columnas_vehiculos) \
    .agg(F.count('*').alias('vehiculos_involucrados')) \
    .orderBy(F.desc('vehiculos_involucrados'))  

vehiculos.show(10)
vehiculos.count()

+-------+-------+-------+--------+---------+---------+--------+-----------+--------+------+--------+----------------------+
|año_ocu|mes_ocu|día_ocu|hora_ocu|depto_ocu|mupio_ocu|zona_ocu|día_sem_ocu|tipo_eve|g_hora|g_hora_5|vehiculos_involucrados|
+-------+-------+-------+--------+---------+---------+--------+-----------+--------+------+--------+----------------------+
|   2018|      3|      1|      13|        1|      117|      99|          4|       1|     3|       2|                    26|
|   2015|      8|     22|       8|        8|      802|      99|          6|       1|     2|       1|                    12|
|   2020|      6|     24|      11|        1|      101|      25|          3|       1|     2|       1|                    12|
|   2020|     12|     23|      15|        1|      115|      99|          3|       1|     3|       2|                    11|
|   2021|     12|     12|      19|        5|      511|      99|          7|       1|     4|       3|                    11|
|   2023

64270

514 accidentes se están agrupando incorrectamente. 

#### Con fallecidos

In [30]:
df = df_fallecidos.withColumn(
    'fall_les_int',
    F.when(F.col('fall_les').rlike('^[0-9]+$'), F.col('fall_les').cast('int')) \
     .when(F.lower(F.col('fall_les')).like('%fallec%'), F.lit(1)) \
     .when(F.lower(F.col('fall_les')).like('%lesion%'), F.lit(2)) \
     .otherwise(None)
)

group_cols = [
    'año_ocu', 'mes_ocu', 'día_ocu', 'hora_ocu',
    'depto_ocu', 'mupio_ocu', 'zona_ocu', 'día_sem_ocu', 'tipo_eve', 
    'g_hora', 'g_hora_5'
]

# 4) Agrupar y agregar
fallecidos = df.groupBy(group_cols).agg(
    # cuenta total de filas (vehículos reportados)
    F.count('*').alias('filas_reportadas'),
    # cuenta de firmas de vehículo distintas (más robusto frente a duplicados)
    #F.countDistinct('veh_sig').alias('vehiculos_involucrados'),
    # conteos condicionales por tipo de fall_les
    F.sum(F.when(F.col('fall_les_int') == 1, 1).otherwise(0)).alias('n_fallecidos'),
    F.sum(F.when(F.col('fall_les_int') == 2, 1).otherwise(0)).alias('n_lesionados')
)

# ordenar para ver los casos grandes
fallecidos = fallecidos.orderBy(F.desc('filas_reportadas'))

fallecidos.show(20, truncate=False)
fallecidos.count()

+-------+-------+-------+--------+---------+---------+--------+-----------+--------+------+--------+----------------+------------+------------+
|año_ocu|mes_ocu|día_ocu|hora_ocu|depto_ocu|mupio_ocu|zona_ocu|día_sem_ocu|tipo_eve|g_hora|g_hora_5|filas_reportadas|n_fallecidos|n_lesionados|
+-------+-------+-------+--------+---------+---------+--------+-----------+--------+------+--------+----------------+------------+------------+
|2016   |3      |28     |5       |7        |705      |99      |1          |4       |1     |1       |57              |24          |33          |
|2019   |9      |26     |19      |5        |501      |99      |4          |3       |4     |3       |57              |6           |51          |
|2016   |5      |30     |5       |6        |601      |99      |1          |4       |1     |1       |50              |1           |49          |
|2023   |4      |8      |22      |9        |901      |99      |6          |1       |4     |3       |49              |4           |45    

59025

#### Join

In [36]:
columnas_union = [
    'año_ocu', 'mes_ocu', 'día_ocu',
    'depto_ocu', 'mupio_ocu', 'zona_ocu',
    'día_sem_ocu', 'tipo_eve', 'g_hora', 'g_hora_5'
]

# Join de manera segura
df_final = fallecidos.join(
    vehiculos,
    on=columnas_union,
    how='inner'   # o 'full_outer' para conservar todos los registros
)

# Si prefieres conservar solo los accidentes que aparecen en ambos:
# how='inner'

# Reemplazar nulls por 0 para conteos
df_final = df_final.fillna({
    'vehiculos_involucrados': 0,
    'n_fallecidos': 0,
    'n_lesionados': 0
})

df_final.show(10, truncate=False)
df_final.count()

+-------+-------+-------+---------+---------+--------+-----------+--------+------+--------+--------+----------------+------------+------------+--------+----------------------+
|año_ocu|mes_ocu|día_ocu|depto_ocu|mupio_ocu|zona_ocu|día_sem_ocu|tipo_eve|g_hora|g_hora_5|hora_ocu|filas_reportadas|n_fallecidos|n_lesionados|hora_ocu|vehiculos_involucrados|
+-------+-------+-------+---------+---------+--------+-----------+--------+------+--------+--------+----------------+------------+------------+--------+----------------------+
|2015   |2      |8      |5        |503      |99      |7          |1       |2     |1       |11      |2               |2           |0           |10      |2                     |
|2015   |4      |10     |2        |203      |99      |5          |1       |4     |3       |21      |2               |2           |0           |21      |3                     |
|2015   |4      |27     |4        |401      |99      |1          |1       |4     |3       |21      |1               |0  

57958

## Análisis de correlaciones

In [38]:
df_final.show()

+-------+-------+-------+---------+---------+--------+-----------+--------+------+--------+--------+----------------+------------+------------+--------+----------------------+
|año_ocu|mes_ocu|día_ocu|depto_ocu|mupio_ocu|zona_ocu|día_sem_ocu|tipo_eve|g_hora|g_hora_5|hora_ocu|filas_reportadas|n_fallecidos|n_lesionados|hora_ocu|vehiculos_involucrados|
+-------+-------+-------+---------+---------+--------+-----------+--------+------+--------+--------+----------------+------------+------------+--------+----------------------+
|   2015|      2|      8|        5|      503|      99|          7|       1|     2|       1|      11|               2|           2|           0|      10|                     2|
|   2015|      4|     10|        2|      203|      99|          5|       1|     4|       3|      21|               2|           2|           0|      21|                     3|
|   2015|      4|     27|        4|      401|      99|          1|       1|     4|       3|      21|               1|   

## Reducción de dimensionalidad PCA

## Segmentación KMeans

# Modelado supervisado para predicción

## Columna Severidad

## Train y Test

## Pipeline de clasificación

## Modelos adicionales

### Regresión logística

### Decision Tree

### Discusión

## Regresión lineal

## Recarga del modelo

## Análisis de la regresión

### Gráficas

### Discusión